This notebook show how to generate a pandas dataframe from several JSON files with tweets captures using the Stream API. 
Particularly, two dataframes are generated:
* A dataframe with tweet and user info
* A dataframe with location info of the tweets

# Configuration

* Connection with Master notebook is configures
* DIfferent variables are set

In [1]:
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False
try: IS_MASTER
except: IS_MASTER = False
# The code below will only run if it's NOT being called from MASTER notebook
if not IS_MASTER:
    # targetFiles = 'HealthyFood'
    # targetFiles = 'UnhealthyFood'
    targetFiles = 'GenericTweets'
    DATA_DIR = './data/temp/' # 
#The code expects a folder (targetFiles) with the next tree structure inside:
# - Name of captured dataset (JSON files)
# |- Year
#  |- Month
#   |- Day
#    |- JSON files
RAW_JSON_DIR = f'./data/JSONFiles/{targetFiles}' #Path to the root folder of JSON files containing tweets
RAW_CSV_DIR = f'./data/CSVFiles/'
PROCESSED_CSV_FILE = f'./{RAW_CSV_DIR}/{targetFiles}-DataFrame.csv'
PROCESSED_LOCATION_CSV_FILE = f'./{RAW_CSV_DIR}/{targetFiles}-LocationsDataframe.csv' # use "small data" here

In [2]:
#Libraries used in the notebook
import csv
import json
import numpy as np
import pandas as pd
import glob
import os
import ntpath
import pickle
import shutil


# 1: Transform the json files to csv

In [3]:
#We get a list with all target JSON files

files = glob.glob(f'{RAW_JSON_DIR}/**/*.json', recursive=True)
files.sort()
filenames = []
for file in files:
    filename=os.path.splitext(file)[0]
    filename = ntpath.basename(filename)
    filenames.append(filename)

filenames

['2019_11_13_13_36_41-Copy1',
 '2019_11_13_14_36_42-Copy1',
 '2019_11_13_15_36_43-Copy1',
 '2019_11_13_16_36_43-Copy1',
 '2019_11_13_17_36_43-Copy1',
 '2019_11_13_18_36_44-Copy1']

In [4]:
# Convert JSON to CSV files
# It may take some time
i = 0
NEW_FILES = 0
outdir = RAW_CSV_DIR + 'temp/'
if not os.path.exists(outdir):
    os.mkdir(outdir)
failed_files = []                              #JSON files in this list need a ']' at the end due to interrupts during the stream capture

for file in files:
    outname = '{}'.format(filenames[i])+'.csv'
    fullname = os.path.join(outdir, outname)
    i+=1

    if not os.path.exists(fullname):
        print('Creating {}'.format(fullname))
        with open(file, encoding='utf-8-sig') as f_input:
            try:
                df = pd.read_json(f_input)
            except:
                failed_files.append(f_input)
            
        df.to_csv(fullname , index=False)
        NEW_FILES +=1

print('{} new csv files has been created'.format(NEW_FILES))

Creating ./data/CSVFiles/temp/2019_11_13_13_36_41-Copy1.csv
Creating ./data/CSVFiles/temp/2019_11_13_14_36_42-Copy1.csv
Creating ./data/CSVFiles/temp/2019_11_13_15_36_43-Copy1.csv
Creating ./data/CSVFiles/temp/2019_11_13_16_36_43-Copy1.csv
Creating ./data/CSVFiles/temp/2019_11_13_17_36_43-Copy1.csv
Creating ./data/CSVFiles/temp/2019_11_13_18_36_44-Copy1.csv
6 new csv files has been created


In [5]:
failed_files

[]

# 2: Create a dataframe

In [21]:
files = glob.glob(f'{RAW_CSV_DIR}temp/*.csv')
files.sort()
files

['./data/CSVFiles//smallData-DataFrame.csv']

In [24]:
# Join all csv files in one dataframe in case there are new files
#TODO:It could have less computational cost if we only join new csv files

if NEW_FILES != 0:

    df = pd.DataFrame()
    print(df.shape)

    li = []

    for filename in files:
        df_temp = pd.read_csv(filename, index_col=None, header=0, lineterminator='\n')
        df_temp = df_temp[['coordinates','created_at','geo','id','id_str','lang','place',
                           'text','user', 'extended_tweet', 'truncated', 'extended_entities']]
        li.append(df_temp)

    df = pd.concat(li, axis=0, ignore_index=True)
    print(df.shape)

    shutil.rmtree(outdir)
    
    #Filter RTs

    df = df[df['text'].str.contains("RT @") == False]
    df = df.reset_index()
    
    print(df.shape)

(0, 0)
(13525, 12)
(13525, 13)


In [25]:
#Get full text from extended tweet
def getFullText(truncated, extended_tweet):
    if truncated:
        a = extended_tweet.replace("'", '"')
        try:
            b = json.loads(a)
            res = b['full_text']
        except:
            res = None
        return res

In [26]:
if NEW_FILES != 0:
    df['full_text'] = df.apply(lambda x: getFullText(x['truncated'], x['extended_tweet']) , axis=1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13525 entries, 0 to 13524
Data columns (total 14 columns):
index                13525 non-null int64
coordinates          50 non-null object
created_at           13525 non-null object
geo                  50 non-null object
id                   13525 non-null float64
id_str               13525 non-null float64
lang                 13525 non-null object
place                525 non-null object
text                 13525 non-null object
user                 13525 non-null object
extended_tweet       3312 non-null object
truncated            13525 non-null float64
extended_entities    653 non-null object
full_text            2780 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 1.4+ MB


In [28]:
def getText2Proc(fullText, text):
    if fullText is not None:
        return (fullText)
    else:
        return (text)

In [29]:
df['text2Proc'] = df.apply(lambda x: getText2Proc(x['full_text'], x['text']) , axis=1)
df.head()


,index,coordinates,created_at,geo,id,id_str,lang,place,text,user,extended_tweet,truncated,extended_entities,full_text,text2Proc
0,0,NaN,2019-11-07 15:51:39.000,NaN,1.192470e+18,1.192470e+18,es,NaN,Nunca voy a entender por qué #Bienvenidos13 si...,"{'id': 284654792, 'id_str': '284654792', 'name...",{'full_text': 'Nunca voy a entender por qué #B...,1.0,NaN,Nunca voy a entender por qué #Bienvenidos13 si...,Nunca voy a entender por qué #Bienvenidos13 si...
1,1,NaN,2019-11-07 15:52:52.000,NaN,1.192470e+18,1.192470e+18,es,NaN,@MalenaAubone NooOOoOOo man voy a comprar hela...,"{'id': 3029655005, 'id_str': '3029655005', 'na...",NaN,0.0,NaN,None,@MalenaAubone NooOOoOOo man voy a comprar hela...
2,2,NaN,2019-11-07 15:53:49.000,NaN,1.192470e+18,1.192470e+18,es,NaN,"Estamos en una parri con mi viejo, al costado ...","{'id': 602970512, 'id_str': '602970512', 'name...",{'full_text': 'Estamos en una parri con mi vie...,1.0,NaN,"Estamos en una parri con mi viejo, al costado ...","Estamos en una parri con mi viejo, al costado ..."
3,3,NaN,2019-11-07 15:54:03.000,NaN,1.192470e+18,1.192470e+18,es,NaN,No nos extinguimos de pedo,"{'id': 1726824211, 'id_str': '1726824211', 'na...",NaN,0.0,NaN,None,No nos extinguimos de pedo
4,4,NaN,2019-11-07 15:55:01.000,NaN,1.192471e+18,1.192471e+18,es,NaN,Desde las 11am. Que pienso en el helado que es...,"{'id': 185476629, 'id_str': '185476629', 'name...",NaN,0.0,NaN,None,Desde las 11am. Que pienso en el helado que es...


In [30]:
outdir = RAW_CSV_DIR
if not os.path.exists(outdir):
    os.mkdir(outdir)
df.to_csv(PROCESSED_CSV_FILE, index=False)

# 3: Get tweet location

In [31]:
# Get user location
users = df.user.values
locations = []

if NEW_FILES != 0:
    for user in users:
        keyword = "'location':"
        before_keyword, keyword, after_keyword = user.partition(keyword)
        my_string = after_keyword
        keyword = "'url':"
        before_keyword, keyword, after_keyword = my_string.partition(keyword)
        user_location = before_keyword.replace(',','').replace("\'",'').replace('\"','').strip()
        locations.append(user_location)
    df = df.assign(user_location = locations)
    df = df.replace('None', np.NaN)

In [32]:
# Get tweet location

if NEW_FILES != 0:
    place = df.place

    longitude = []
    latitude = []
    country = []
    city = []
    n=0
    for i in place.values:
        a = ''
        if not isinstance(i, float):
            a = i.replace("'", '"')

        try:
            b = json.loads(a)
            n = n + 1
            longitude.append(b['bounding_box']['coordinates'][0][0][0])
            latitude.append(b['bounding_box']['coordinates'][0][0][1])
            country.append(b['country'])
            city.append(b['name'])
        except ValueError:
            longitude.append(np.NaN)
            latitude.append(np.NaN)
            country.append(np.NaN)
            city.append(np.NaN)
            continue
    df['Country'] = country
    df['City'] = city
    df['Longitude'] = longitude
    df['Latitude'] = latitude
    df['Coordinates'] = list(zip(df.Longitude, df.Latitude))

    df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13525 entries, 0 to 13524
Data columns (total 21 columns):
index                13525 non-null int64
coordinates          50 non-null object
created_at           13525 non-null object
geo                  50 non-null object
id                   13525 non-null float64
id_str               13525 non-null float64
lang                 13525 non-null object
place                525 non-null object
text                 13525 non-null object
user                 13525 non-null object
extended_tweet       3312 non-null object
truncated            13525 non-null float64
extended_entities    653 non-null object
full_text            2780 non-null object
text2Proc            13525 non-null object
user_location        9376 non-null object
Country              524 non-null object
City                 524 non-null object
Longitude            524 non-null float64
Latitude             524 non-null float64
Coordinates          13525 non-null object
dtype

In [33]:
df_original = df


### Create a dataframe with locations for statistical analysis

In [34]:
df = df[['Country', 'City', 'Longitude', 'Latitude']][df['Country'].notna()]
df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
df


,Country,City,Longitude,Latitude,Coordinates
39,España,Pamplona,-1.694620,42.789017,"(-1.69462, 42.789017)"
54,México,Morelia,-101.508271,19.442976,"(-101.508271, 19.442976)"
75,México,Benito Juárez,-99.191996,19.357102,"(-99.191996, 19.357102)"
96,España,Barcelona,2.052477,41.317048,"(2.052477, 41.317048)"
196,Venezuela,Maracaibo,-71.692291,10.596120,"(-71.692291, 10.596119999999999)"
205,Honduras,Honduras,-89.350792,12.984911,"(-89.350792, 12.984911)"
217,Venezuela,Petare,-66.847406,10.457348,"(-66.847406, 10.457348)"
221,España,Madrid,-3.889005,40.312071,"(-3.889005, 40.312071)"
243,Argentina,Posadas,-55.959513,-27.459170,"(-55.959513, -27.45917)"
262,Botswana,Botswana,19.997675,-26.907035,"(19.997675, -26.907035)"


In [35]:
df.to_csv(PROCESSED_LOCATION_CSV_FILE, index=False)

In [36]:
#get tweets from Spain
df = df_original
df.loc[df['Country'] == "España", 'Country'] = 'Spain'
df.loc[df['Country'] == "Espanya", 'Country'] = 'Spain'
df_spain = df = df[df.Country == 'Spain']
df_spain = df_spain.reset_index()
df_spain

,level_0,index,coordinates,created_at,geo,id,id_str,lang,place,text,...,truncated,extended_entities,full_text,text2Proc,user_location,Country,City,Longitude,Latitude,Coordinates
0,39,39,"{'type': 'Point', 'coordinates': [-1.65834, 42...",2019-11-07 17:14:08.000,"{'type': 'Point', 'coordinates': [42.81688, -1...",1.192490e+18,1.192490e+18,es,"{'id': '08eb5d76dc74e0e9', 'url': 'https://api...","Esta es la propuesta de El Ronqueo, Panplona, ...",...,1.0,NaN,"Esta es la propuesta de El Ronqueo, Panplona, ...","Esta es la propuesta de El Ronqueo, Panplona, ...",Pamplona,Spain,Pamplona,-1.694620,42.789017,"(-1.69462, 42.789017)"
1,96,96,NaN,2019-11-07 18:44:43.000,NaN,1.192513e+18,1.192513e+18,es,"{'id': '1a27537478dd8e38', 'url': 'https://api...",@JoanTarda @Esquerra_ERC Es que ese es el prob...,...,1.0,NaN,@JoanTarda @Esquerra_ERC Es que ese es el prob...,@JoanTarda @Esquerra_ERC Es que ese es el prob...,Barcelona Catalunya,Spain,Barcelona,2.052477,41.317048,"(2.052477, 41.317048)"
2,221,221,NaN,2019-11-07 19:39:44.000,NaN,1.192527e+18,1.192527e+18,es,"{'id': '206c436ce43a43a3', 'url': 'https://api...",Muy interesante este artículo sobre el efecto ...,...,1.0,NaN,Muy interesante este artículo sobre el efecto ...,Muy interesante este artículo sobre el efecto ...,Madrid,Spain,Madrid,-3.889005,40.312071,"(-3.889005, 40.312071)"
3,337,337,NaN,2019-11-08 15:31:15.000,NaN,1.192827e+18,1.192827e+18,es,"{'id': '8c86b8b4cb716103', 'url': 'https://api...",Pues el chocolate navideño del Starbucks está ...,...,0.0,NaN,None,Pues el chocolate navideño del Starbucks está ...,Sevilla - Dorne,Spain,Sevilla,-6.028430,37.313613,"(-6.02843, 37.313613)"
4,369,369,"{'type': 'Point', 'coordinates': [-6.21495025,...",2019-11-08 16:26:56.000,"{'type': 'Point', 'coordinates': [36.61219203,...",1.192841e+18,1.192841e+18,es,"{'id': '3f7a6158324f3e9f', 'url': 'https://api...",Contigo al fin del mundo...y vi ellas día deli...,...,1.0,NaN,Contigo al fin del mundo...y vi ellas día deli...,Contigo al fin del mundo...y vi ellas día deli...,El Puerto de Santa Maria,Spain,El Puerto de Santa María,-6.323065,36.525759,"(-6.323065, 36.525759)"
5,551,551,NaN,2019-11-08 21:26:08.000,NaN,1.192916e+18,1.192916e+18,es,"{'id': '16d7e4bdce732b26', 'url': 'https://api...",@WilliamHillES #miapuesta En el 76sixers vs Nu...,...,0.0,NaN,None,@WilliamHillES #miapuesta En el 76sixers vs Nu...,Lloret de Mar,Spain,Lloret de Mar,2.775933,41.686311,"(2.775933, 41.686311)"
6,690,690,NaN,2019-11-09 00:57:50.000,NaN,1.192970e+18,1.192970e+18,es,"{'id': '3afc29d1fe76301f', 'url': 'https://api...","Cómo necesito una noche de verano , con vino, ...",...,0.0,NaN,None,"Cómo necesito una noche de verano , con vino, ...",San Rafael - Mendoza,Spain,Zaragoza,-1.174093,41.451835,"(-1.174093, 41.451835)"
7,905,905,NaN,2019-11-09 09:06:59.000,NaN,1.193093e+18,1.193093e+18,es,"{'id': '84809564179ffb21', 'url': 'https://api...",@GalaRomani Como sustituto de liar pitillos no...,...,0.0,NaN,None,@GalaRomani Como sustituto de liar pitillos no...,euskal herria-basque country,Spain,Barakaldo,-3.048774,43.238030,"(-3.048774, 43.23803)"
8,1642,1642,"{'type': 'Point', 'coordinates': [-3.8647, 40....",2019-11-10 11:28:27.000,"{'type': 'Point', 'coordinates': [40.3218, -3....",1.193491e+18,1.193491e+18,es,"{'id': 'd84973286cfd241b', 'url': 'https://api...",💀💀💀\n⚠️PACK INDIVISIBLE!⚠️\nNunca fuimos igual...,...,1.0,NaN,💀💀💀\n⚠️PACK INDIVISIBLE!⚠️\nNunca fuimos igual...,💀💀💀\n⚠️PACK INDIVISIBLE!⚠️\nNunca fuimos igual...,NaN,Spain,Móstoles,-3.945389,40.285747,"(-3.945389, 40.285747)"
9,1684,1684,NaN,2019-11-10 13:40:23.000,NaN,1.193524e+18,1.193524e+18,es,"{'id': '38795862f9482aaa', 'url': 'https://api...",Tengo antojo de tarta de queso,...,0.0,NaN,None,Tengo antojo de tarta de queso,Los Villares Jaén,Spain,Los Villares,-3.893182,37.634566,"(-3.893182, 37.634566)"


In [37]:
# Tweets with extended entities (multimedia) from Spain
df_spain[df_spain['extended_entities'].notna()]

,level_0,index,coordinates,created_at,geo,id,id_str,lang,place,text,...,truncated,extended_entities,full_text,text2Proc,user_location,Country,City,Longitude,Latitude,Coordinates
65,10496,10496,NaN,2019-12-02 21:35:09.000,NaN,1.201616e+18,1.201616e+18,es,"{'id': '507a015cba6ef518', 'url': 'https://api...",Por favor depositad un kebab en mi boca https:...,...,0.0,"{'media': [{'id': 1201615794028843009, 'id_str...",None,Por favor depositad un kebab en mi boca https:...,Málaga España,Spain,Málaga,-4.587928,36.635585,"(-4.587928, 36.635585)"
